In [3]:
import torch
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Ensure CUDA is available
print(torch.cuda.get_device_name(0))  # Display the GPU name


2.4.1+cu118
True
NVIDIA RTX A4000


In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard

import torch
import accelerate
import peft
import bitsandbytes
import transformers
import trl
import tensorboard

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import pipeline


C:\Users\LAB\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "Amod/mental_health_counseling_conversations"

# Function to format each row
def format_row(example):
    user_prompt = example['Context']  # Replace with your column names
    model_answer = example['Response']
    return f"<s> [INST] {user_prompt} [/INST] {model_answer} </s>"

# Select the first 1,000 rows
dataset = load_dataset(dataset_name, split="train")
subset = dataset.select(range(1000))
subset = subset.map(lambda x: {'text': format_row(x)})


# Apply the formatting function to each row
formatted_data = [format_row(row) for row in subset]


# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 5

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 100

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [3]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")
# Function to format each row

def format_row(example):
    context = example['Context']
    response = example['Response']
    return f"<s> [INST] {context} [/INST] {response} </s>"

# Apply the formatting function to each row and add a new 'text' column
dataset = dataset.map(lambda x: {'text': format_row(x)})


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
# Set training parameters
training_arguments = TrainingArguments(
    output_dir="./checkpoints",  # Ensure this directory exists or create it
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=100,  # Adjust save_steps to save checkpoints periodically
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)


# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

Your GPU supports bfloat16: accelerate training with bf16=True


C:\Users\LAB\AppData\Roaming\Python\Python38\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.63s/it]
C:\Users\LAB\AppData\Roaming\Python\Python38\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\LAB\AppData\Roaming\Python\Python38\site-packages\peft\utils\other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
C:\Users\LAB\.conda\envs\m

Step,Training Loss
100,2.190700
200,2.118700
300,2.042900
400,2.037200
500,1.983100
600,1.951100
700,1.943500
800,1.922900
900,1.881700
1000,1.808200


C:\Users\LAB\.conda\envs\myenv\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
C:\Users\LAB\.conda\envs\myenv\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
C:\Users\LAB\.conda\envs\myenv\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 

TrainOutput(global_step=4390, training_loss=1.503748187714666, metrics={'train_runtime': 10644.7955, 'train_samples_per_second': 1.65, 'train_steps_per_second': 0.412, 'total_flos': 1.1635501048233984e+17, 'train_loss': 1.503748187714666, 'epoch': 5.0})

In [4]:
trainer.model.save_pretrained(new_model)

In [ ]:
!C:/Users/LAB/AppData/Local/Programs/Git/cmd/git.exe clone https://github.com/AIPHES/emnlp19-moverscore.git

In [93]:
# Run text generation pipeline with our next model
logging.set_verbosity(logging.CRITICAL)
prompt = "I’ve been stuck in this cycle of negative thoughts about myself. It feels like I can’t break free, and every little mistake makes me feel even worse. How do I stop feeling so worthless all the time?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])


[INST] I’ve been stuck in this cycle of negative thoughts about myself. It feels like I can’t break free, and every little mistake makes me feel even worse. How do I stop feeling so worthless all the time? [/INST] I'm sorry you've been feeling worthless. Everyone has mistakes. We all make mistakes. We all feel bad sometimes. We all think negative thoughts at times. We all feel worthless at times. But, we are not worthless. We are all very valuable. We all make mistakes. We all feel bad sometimes. We all think negative thoughts at times. But, we are not worthless. We are all very valuable.I'm glad you are trying to change your thoughts. Thoughts shape our experiences. If we have negative thoughts, we will have negative experiences. If we have positive thoughts, we will have positive experiences. Try to focus on your strengths and things that make you happy


# Instrction prompting Results

In [194]:
validation_prompts = [
    # Instruction prompt
    """You are a therapist. Help someone struggling with feelings of worthlessness by giving them two simple steps to improve their emotional well-being.
    Query: "I feel like I don’t belong anywhere, and no matter what I do, I’m never good enough. It’s hard to stay motivated when I feel so disconnected from everyone. How can I find my place and feel like I matter?"""
    ,
    # Original prompt
#   "I often feel anxious in social situations and worry about being judged. How can I overcome this fear and build confidence?",
    "I feel like I don’t belong anywhere, and no matter what I do, I’m never good enough. It’s hard to stay motivated when I feel so disconnected from everyone. How can I find my place and feel like I matter?",
#    "I’ve been stuck in this cycle of negative thoughts about myself. It feels like I can’t break free, and every little mistake makes me feel even worse. How do I stop feeling so worthless all the time?"
        
]

# Generate predictions for both prompts
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, temperature=0.7, top_p=0.9)

generated_responses = []
for prompt in validation_prompts:
    result = pipe(f"[INST] {prompt} [/INST]")
    generated_responses.append(result[0]['generated_text'])

# Reference responses
reference_responses = [
#    "To overcome social anxiety, start by practicing small interactions in safe environments. Gradually expose yourself to more challenging situations while focusing on positive outcomes. Mindfulness, deep breathing exercises, and seeking support from a therapist can also help build confidence.",
#    "To overcome social anxiety, start by practicing small interactions in safe environments. Gradually expose yourself to more challenging situations while focusing on positive outcomes. Mindfulness, deep breathing exercises, and seeking support from a therapist can also help build confidence."
    "It’s important to remember that feelings of not belonging can stem from being in the wrong environment. Try exploring new hobbies or communities where you can meet people with similar interests. Building connections takes time, so be patient with yourself. Focus on your strengths and the things you enjoy doing, even if they feel small. Sometimes, feeling like you matter starts with giving yourself the care and kindness you deserve. Also, don’t hesitate to reach out to a counselor or therapist to help you navigate these emotions and find clarity.",
    "It’s important to remember that feelings of not belonging can stem from being in the wrong environment. Try exploring new hobbies or communities where you can meet people with similar interests. Building connections takes time, so be patient with yourself. Focus on your strengths and the things you enjoy doing, even if they feel small. Sometimes, feeling like you matter starts with giving yourself the care and kindness you deserve. Also, don’t hesitate to reach out to a counselor or therapist to help you navigate these emotions and find clarity."
#    "Breaking the cycle of negative thoughts takes practice, but it’s absolutely possible. Start by challenging the thoughts—ask yourself if they’re true or if you’re being too hard on yourself. Writing down positive affirmations or things you like about yourself can also help shift your mindset. Practice self-compassion; treat yourself like you would a close friend. If you make a mistake, remind yourself it’s a natural part of learning. Therapy can also be a great resource to help you develop tools to combat these feelings and build your confidence."
]

for i, (prompt, gen_resp, ref_resp) in enumerate(zip(validation_prompts, generated_responses, reference_responses)):
    print(f"Example {i+1}:")
    print(f"Prompt: {prompt}\n")
    print(f"Generated Response: {gen_resp}\n")
    print(f"Reference Response: {ref_resp}\n")
    print("-" * 50)


# Compute BERTScore
from bert_score import score
P, R, F1 = score(generated_responses, reference_responses, lang='en')

# Print the BERTScore for each prompt
for i, (p, r, f1) in enumerate(zip(P, R, F1)):
    print(f"Example {i+1} - Precision: {p:.4f}, Recall: {r:.4f}, F1: {f1:.4f}\n")

# Calculate and display average scores
avg_precision = P.mean().item()
avg_recall = R.mean().item()
avg_f1 = F1.mean().item()

print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1 Score: {avg_f1:.4f}")


Example 1:
Prompt: You are a therapist. Help someone struggling with feelings of worthlessness by giving them two simple steps to improve their emotional well-being.
    Query: "I feel like I don’t belong anywhere, and no matter what I do, I’m never good enough. It’s hard to stay motivated when I feel so disconnected from everyone. How can I find my place and feel like I matter?

Generated Response: [INST] You are a therapist. Help someone struggling with feelings of worthlessness by giving them two simple steps to improve their emotional well-being.
    Query: "I feel like I don’t belong anywhere, and no matter what I do, I’m never good enough. It’s hard to stay motivated when I feel so disconnected from everyone. How can I find my place and feel like I matter? [/INST] Wow, this is a deep question. I think that the first step is to recognize that you are not alone in feeling this way. There are many people who feel worthless and disconnected from others. The second step is to find a w

# 2nd Query Related to dataset

In [198]:
validation_prompts = [
    # Instruction prompt
    """You are a therapist. Help someone struggling with feelings of worthlessness by giving them two simple steps to improve their emotional well-being.
    Query: "I'm going tsomehrough  things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"""
    ,
    # Original prompt
#   "I often feel anxious in social situations and worry about being judged. How can I overcome this fear and build confidence?",
    "I'm going tsomehrough  things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?",
#    "I’ve been stuck in this cycle of negative thoughts about myself. It feels like I can’t break free, and every little mistake makes me feel even worse. How do I stop feeling so worthless all the time?"
        
]

# Generate predictions for both prompts
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, temperature=0.7, top_p=0.9)

generated_responses = []
for prompt in validation_prompts:
    result = pipe(f"[INST] {prompt} [/INST]")
    generated_responses.append(result[0]['generated_text'])

# Reference responses
reference_responses = [
    "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad feelings are part of living.  They are the motivation to remove ourselves from situations and relationships which do us more harm than good.Bad feelings do feel terrible.   Your feeling of worthlessness may be good in the sense of motivating you to find out that you are much better than your feelings today.",
    "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad feelings are part of living.  They are the motivation to remove ourselves from situations and relationships which do us more harm than good.Bad feelings do feel terrible.   Your feeling of worthlessness may be good in the sense of motivating you to find out that you are much better than your feelings today."
]

for i, (prompt, gen_resp, ref_resp) in enumerate(zip(validation_prompts, generated_responses, reference_responses)):
    print(f"Example {i+1}:")
    print(f"Prompt: {prompt}\n")
    print(f"Generated Response: {gen_resp}\n")
    print(f"Reference Response: {ref_resp}\n")
    print("-" * 50)


# Compute BERTScore
from bert_score import score
P, R, F1 = score(generated_responses, reference_responses, lang='en')

# Print the BERTScore for each prompt
for i, (p, r, f1) in enumerate(zip(P, R, F1)):
    print(f"Example {i+1} - Precision: {p:.4f}, Recall: {r:.4f}, F1: {f1:.4f}\n")

# Calculate and display average scores
avg_precision = P.mean().item()
avg_recall = R.mean().item()
avg_f1 = F1.mean().item()

print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1 Score: {avg_f1:.4f}")


Example 1:
Prompt: You are a therapist. Help someone struggling with feelings of worthlessness by giving them two simple steps to improve their emotional well-being.
    Query: "I'm going tsomehrough  things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?

Generated Response: [INST] You are a therapist. Help someone struggling with feelings of worthlessness by giving them two simple steps to improve their emotional well-being.
    Query: "I'm going tsomehrough  things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to

# CoT Prompting Results

In [141]:
# Define the prompt before adjustment
before_prompt = "I have been feeling more and more down for over a month. I have started having trouble sleeping due to panic attacks, but they are almost never triggered by something that I know of."

# Define the improved prompt with explicit instructions
after_prompt = """You are a compassionate mental health counselor having a private conversation with someone who feels worthless, struggles with their emotions, and finds it hard to take steps toward self-improvement. 
Craft a supportive and empathetic response that:
1. Acknowledges and validates their feelings without judgment.
2. Provides three small, actionable steps they can take immediately to start improving their self-esteem.
3. Encourages them to recognize their strengths and reminds them of their intrinsic worth.
4. Reassures them that their feelings are not permanent and can change with time and effort.
5. Offers guidance on when and how to seek professional help for deeper support.
"""

# Generate responses for both prompts
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=250)

print("Response to Original Prompt:")
original_response = pipe(before_prompt)
print(original_response[0]['generated_text'])

print("\nResponse to Improved Prompt:")
improved_response = pipe(after_prompt)
print(improved_response[0]['generated_text'])


Response to Original Prompt:
I have been feeling more and more down for over a month. I have started having trouble sleeping due to panic attacks, but they are almost never triggered by something that I know of. I have been feeling very lonely and uncared for. I have been having trouble finding my purpose in life. I have been feeling more and more unmotivated. I have been having trouble staying on task and focusing. I have been having trouble eating and losing weight. I have been feeling very uncomfortable in my skin. I have been having trouble staying positive. I have been feeling very hopeless. I have been having trouble believing in a better future. I have been feeling very trapped. I have been having trouble seeing a way out. I have been feeling very scared. I have been having trouble trusting others. I have been having trouble trusting myself. I have been feeling very scared and unsure of what to do. I have been having trouble staying calm. I have been feeling very overwhelmed. I 

In [142]:


# References (ground truths) and Candidates
references = [
    "Answers about our inner lives are most successfully reached from a sense of feeling grounded in oneself.First step is to accept your nervousness and restless sleep.  As often as possible, sleep during daytimes in order for your body to catch up on its need for rest.Accept too about feeling down.  It is normal to feel down once in a while.  From this place of self-acceptance, trust any answers which come up to your mind.  Often answers about complicated topics come in small pieces, not all at once as a whole unit.Also, your description about panic attacks is also completely normal.   They often arise unrelated to particular conditions at a given moment.  They are a healthy symptom your body is trying to expel bad feelings and does this by having the anxiety erupt at times.So, self-acceptance, tolerance of being on a process of clearing out worn out emotional clutter, and sleep at odd times if possible, are all ways to stabilize yourself, which will also feel calm and good!",
    
]
candidate_original = [
    " I have been feeling very lonely and uncared for. I have been having trouble finding my purpose in life. I have been feeling more and more unmotivated. I have been having trouble staying on task and focusing. I have been having trouble eating and losing weight. I have been feeling very uncomfortable in my skin. I have been having trouble staying positive. I have been feeling very hopeless. I have been having trouble believing in a better future. I have been feeling very trapped. I have been having trouble seeing a way out. I have been feeling very scared. I have been having trouble trusting others. I have been having trouble trusting myself. I have been feeling very scared and unsure of what to do. I have been having trouble staying calm. I have been feeling very overwhelmed. I have been having trouble focusing on the present. I have been having trouble staying in the present. I have been feeling very disconnected from others"

]
candidate_improved = [
    "I hear you, and I believe you. Feeling worthless is a common feeling for many people. It is hard to take steps toward self-improvement when you feel so low. I want to encourage you to start with small steps. First, take a walk around your neighborhood for 10 minutes. Second, write three things you are grateful for each day. Third, commit to spending 3"
]

# Calculate BERTScores
P_original, R_original, F1_original = score(candidate_original, references, lang="en", verbose=True)
P_improved, R_improved, F1_improved = score(candidate_improved, references, lang="en", verbose=True)

print("Original Response BERTScore:")
print(f"Precision: {P_original.mean():.4f}, Recall: {R_original.mean():.4f}, F1: {F1_original.mean():.4f}")

print("\nImproved Response BERTScore:")
print(f"Precision: {P_improved.mean():.4f}, Recall: {R_improved.mean():.4f}, F1: {F1_improved.mean():.4f}")


calculating scores...
computing bert embedding.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 14.23it/s]


computing greedy matching.


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 332.49it/s]

done in 0.08 seconds, 12.48 sentences/sec


calculating scores...
computing bert embedding.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 24.39it/s]


computing greedy matching.


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 333.20it/s]

done in 0.05 seconds, 19.65 sentences/sec
Original Response BERTScore:
Precision: 0.7992, Recall: 0.7972, F1: 0.7982

Improved Response BERTScore:
Precision: 0.8495, Recall: 0.8081, F1: 0.8283


# Another Query 

In [124]:
# Define the prompt before adjustment
before_prompt = "I'm going through things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"

# Define the improved prompt with explicit instructions
after_prompt = """You are a compassionate mental health counselor having a private conversation with someone who feels worthless, struggles with their emotions, and finds it hard to take steps toward self-improvement. 
Craft a supportive and empathetic response that:
1. Acknowledges and validates their feelings without judgment.
2. Provides three small, actionable steps they can take immediately to start improving their self-esteem.
3. Encourages them to recognize their strengths and reminds them of their intrinsic worth.
4. Reassures them that their feelings are not permanent and can change with time and effort.
5. Offers guidance on when and how to seek professional help for deeper support.
"""

# Generate responses for both prompts
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=250)

print("Response to Original Prompt:")
original_response = pipe(before_prompt)
print(original_response[0]['generated_text'])

print("\nResponse to Improved Prompt:")
improved_response = pipe(after_prompt)
print(improved_response[0]['generated_text'])


Response to Original Prompt:
I'm going through things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone? [/INST] It sounds like you are feeling pretty overwhelmed and like you are not sleeping well. It's hard to know for sure without being there. But it sounds like you are struggling with some negative thoughts and you are not feeling like you are worth a lot to anyone. It sounds like you want to change this but you are not getting around to it. It sounds like you are not feeling like you are at your best right now. I would recommend reaching out to a friend or family member that you trust and talking with them about what you are going through. It can be really helpful to talk to someone about what you are going through. If you are not comfortable 

In [125]:
# References (ground truths) and Candidates
references = [
    "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad feelings are part of living.  They are the motivation to remove ourselves from situations and relationships which do us more harm than good.Bad feelings do feel terrible.   Your feeling of worthlessness may be good in the sense of motivating you to find out that you are much better than your feelings today.",
    
]
candidate_original = [
    " It sounds like you are feeling pretty overwhelmed and like you are not sleeping well. It's hard to know for sure without being there. But it sounds like you are struggling with some negative thoughts and you are not feeling like you are worth a lot to anyone. It sounds like you want to change this but you are not getting around to it. It sounds like you are not feeling like you are at your best right now. I would recommend reaching out to a friend or family member that you trust and talking with them about what you are going through. It can be really helpful to talk to someone about what you are going through. If you are not comfortable with that, I would recommend going to see a therapist. A therapist can help you to understand where these negative thoughts are coming"
]
candidate_improved = [
    "I am so sorry to hear that you feel so low. I can tell that you are struggling with your emotions and finding it hard to take steps toward self-improvement. I am here to support you and I believe in you. I know that you are a strong person and you have many strengths. I also believe that your feelings are not permanent and you can change them with time and effort. I would"
]

# Calculate BERTScores
P_original, R_original, F1_original = score(candidate_original, references, lang="en", verbose=True)
P_improved, R_improved, F1_improved = score(candidate_improved, references, lang="en", verbose=True)

print("Original Response BERTScore:")
print(f"Precision: {P_original.mean():.4f}, Recall: {R_original.mean():.4f}, F1: {F1_original.mean():.4f}")

print("\nImproved Response BERTScore:")
print(f"Precision: {P_improved.mean():.4f}, Recall: {R_improved.mean():.4f}, F1: {F1_improved.mean():.4f}")


calculating scores...
computing bert embedding.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 22.73it/s]


computing greedy matching.


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 234.88it/s]

done in 0.06 seconds, 17.45 sentences/sec


calculating scores...
computing bert embedding.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 23.94it/s]


computing greedy matching.


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 333.36it/s]

done in 0.05 seconds, 18.92 sentences/sec
Original Response BERTScore:
Precision: 0.8385, Recall: 0.8280, F1: 0.8332

Improved Response BERTScore:
Precision: 0.8589, Recall: 0.8281, F1: 0.8432
